## Zadanie 2
### Hubert Kłosowski 242424
### Kamil Małecki 242464

### Potrzebne importy

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from scipy.stats import iqr, zscore

## Wczytanie danych

In [ ]:
def get_data():
    os.chdir('..')
    os.chdir('Zadanie1\\czesc4')
    read = [pd.read_csv(filename, parse_dates=[1]) for filename in os.listdir()]
    os.chdir('..')
    os.chdir('..')
    os.chdir('Zadanie2')
    return read

data = get_data()

## <center>Część 1</center>
### Wykonaj wykresy pudełkowe (ang.box plot), które pozwolą na graficzną identyfikację wartości odstających poprzez przedstawienie kwartyli i wartości skrajnych w danych dotyczących:

### 1.1 liczby nowych zachorowań

In [ ]:
sns.boxplot(data=data[1], y='daily_confirmed', linewidth=2)
plt.title('Wykres pudełkowy liczby nowych zachorowań')
plt.ylabel('Nowe zachorowania')

### 1.2 liczby nowych śmierci

In [ ]:
sns.boxplot(data=data[2], y='daily_deceased', linewidth=2)
plt.title('Wykres pudełkowy liczby nowych śmierci')
plt.ylabel('Nowe śmierci')

### 1.3 liczby nowych szczepień

In [ ]:
sns.boxplot(data=data[3], y='daily_persons_vaccinated', linewidth=2)
plt.title('Wykres pudełkowy liczby nowych szczepień')
plt.ylabel('Nowe szczepienia')

### 1.4 populacji

In [ ]:
sns.boxplot(data=data[0], y='population', linewidth=2, log_scale=10)
plt.title('Wykres pudełkowy populacji w krajach na świecie')
plt.ylabel('Populacja')

### 1.5 PKB

In [ ]:
sns.boxplot(data=data[0], y='gdp_usd', linewidth=2, log_scale=10)
plt.title('Wykres pudełkowy PKB na świecie [USD]')
plt.ylabel('PKB krajów w dolarach')

### 1.6
### średnia długość życia w krajach

In [ ]:
sns.boxplot(data=data[0], y='life_expectancy', linewidth=2)
plt.title('Wykres pudełkowy przewidywanej długości życia na świecie')
plt.ylabel('Przewidywana długości życia')

### PKB na osobę

In [ ]:
sns.boxplot(data=data[0], y='gdp_per_capita_usd', linewidth=2)
plt.title('Wykres pudełkowy PKB na osobę na świecie')
plt.ylabel('Wartość PKB na osobę')

### cena benzyny w USA i UK

In [ ]:
sns.boxplot(data=data[-1], y='Gasoline Prices $/litre', x='CCA3', linewidth=2)
plt.title('Wykres pudełkowy ceny benzyny w USA i UK')
plt.xlabel('Kraj')
plt.ylabel('Cena benzyny $/litr')

### liczba przestępstw z podziałem na jego rodzaj w Południowej Australi

In [ ]:
sns.boxplot(data=data[-3], x='Offence count', y='Offence Type', linewidth=2)
plt.title('Wykres pudełkowy przestępstw w Południowej Australi')

## <center>Część 2</center>
### Usuń wartości odstające dla co najmniej 2 informacji z części 1 poziomu 2. W tym celu użyj dwóch różnych metod obliczania outlierów. Uzasadnij swój wybór. Przeanalizuj otrzymane wyniki. 
<ol>
    <li style='font-size: 20px'>Metoda Z-Score</li>
    <p style='font-size: 20px'>Metoda statystyczna, która mierzy, jak wiele odchyleń standardowych wartość jest oddalona od średniej.<br>Wartości z wysokim Z-Score (np. powyżej 2 lub 3) mogą być uznane za wartości odstające, a następnie usunięte. </p>
    <br>
    <li style='font-size: 20px'>Metoda IQR (ang. InterQuartile Range)</li>
    <p style='font-size: 20px'>Technika oparta na kwartylach, w której oblicza się zakres międzykwartylowy (Q3 - Q1) i wartości odstające są określane jako wartości leżące poza granicami 1,5 IQR poniżej Q1 lub powyżej Q3. </p>
</ol>

In [ ]:
def del_outliers(series, method='zscore'):
    indexes = []
    if method == 'zscore':
        x = zscore(series)
        for index in range(len(x)):
            if -2 <= x[index] >= 2:
                indexes.append(index)
    elif method == 'iqr':
        q1, q3, c = series.quantile(0.25), series.quantile(0.75), iqr(series) * 1.5
        for index, v in series.items():
            if q1 - c <= v >= q3 + c:
                indexes.append(index)
    else:
        raise ValueError('Nie ma takich rzeczy')
    return indexes

data[1].drop(index=del_outliers(data[1]['daily_confirmed']))
data[2].drop(index=del_outliers(data[2]['daily_deceased'], method='iqr'))

### Wykresy pudełkowe po usunięciu <i>outlierów</i>

In [ ]:
sns.boxplot(data=data[1], y='daily_confirmed', linewidth=2)
plt.title('Wykres pudełkowy liczby nowych zachorowań')
plt.ylabel('Nowe zachorowania')

In [ ]:
sns.boxplot(data=data[2], y='daily_deceased', linewidth=2)
plt.title('Wykres pudełkowy liczby nowych śmierci')
plt.ylabel('Nowe śmierci')

## <center>Część 3</center>
### Wykonaj następujące obliczenia:

### 3.1. średnia liczba nowych zachorowań, śmierci i szczepień dla co najmniej 10 wybranych krajów w wybranym miesiącu,

In [ ]:
countries = data[0][['country_name', 'gdp_usd']].sort_values(by='gdp_usd', ascending=False)[:10]  # wybieramy 10 najbogatszych krajów
countries.drop('gdp_usd', axis=1, inplace=True)
countries.reset_index(drop=True, inplace=True)

df_3_1 = pd.concat([data[1][['date', 'daily_confirmed']], data[2]['daily_deceased'].astype(np.int64), data[3]['daily_persons_vaccinated'].astype(np.int64)], axis=1) # dobieramy odpowiednie kolumny z początkowych danych
df_3_1

### 3.2. średnia zmiana liczby zachorowań, śmierci i szczepień dla co najmniej 10 wybranych krajów w wybranym miesiącu,